In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("US_Accidents_Dec21_updated.csv")

In [3]:
# Check how many values are missing in each column
df.isnull().sum()

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                        0
End_Lng                        0
Distance(mi)                   0
Description                    0
Number                   1743911
Street                         2
Side                           0
City                         137
County                         0
State                          0
Zipcode                     1319
Country                        0
Timezone                    3659
Airport_Code                9549
Weather_Timestamp          50736
Temperature(F)             69274
Wind_Chill(F)             469643
Humidity(%)                73092
Pressure(in)               59200
Visibility(mi)             70546
Wind_Direction             73775
Wind_Speed(mph)           157944
Precipitation(in)         549458
Weather_Condition          70636
Amenity   

In [4]:
# Drop unnecessary columns
df = df.drop(columns=['Number', 'Sunrise_Sunset', 'Nautical_Twilight', 'Astronomical_Twilight', 'Street', 'City', 
                      'Zipcode', 'Timezone','ID', 'End_Lat', 'End_Lng', 'Country', 'Airport_Code', 
                      'Weather_Timestamp', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Wind_Direction', 
                      'Wind_Speed(mph)'])

In [5]:
pd.set_option('display.max_columns', None)
df.head()

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Side,County,State,Temperature(F),Visibility(mi),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Civil_Twilight
0,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,R,Franklin,OH,42.1,10.0,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
1,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,0.747,At OH-4/OH-235/Exit 41 - Accident.,R,Montgomery,OH,36.9,10.0,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
2,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,0.055,At I-71/US-50/Exit 1 - Accident.,R,Hamilton,OH,36.0,10.0,0.02,Overcast,False,False,False,False,True,False,False,False,False,False,False,False,False,Night
3,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,0.123,At Dart Ave/Exit 21 - Accident.,R,Summit,OH,39.0,10.0,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
4,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,0.500,At Mitchell Ave/Exit 6 - Accident.,R,Hamilton,OH,37.0,10.0,0.01,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day


In [6]:
precip_missing = df[df['Precipitation(in)'].isnull()]
precip_missing['Weather_Condition'].value_counts()
# Looks like the reason precipitation is missing in some rows is because there was no accumulated precipitation 
# during the time of the accident.

Clear                 172786
Overcast               72357
Mostly Cloudy          72029
Partly Cloudy          51271
Fair                   45103
                       ...  
Small Hail                 1
Wintry Mix / Windy         1
Snow / Windy               1
Heavy Ice Pellets          1
Light Haze                 1
Name: Weather_Condition, Length: 80, dtype: int64

In [7]:
# We can replace the null values in the precipitation column with 0
df['Precipitation(in)'] = df['Precipitation(in)'].fillna(0.0)

In [8]:
# Now we can drop the rest of the columns with missing values.
df = df.dropna()
df.isnull().sum()
# Voila! No more missing values, as you can see.

Severity             0
Start_Time           0
End_Time             0
Start_Lat            0
Start_Lng            0
Distance(mi)         0
Description          0
Side                 0
County               0
State                0
Temperature(F)       0
Visibility(mi)       0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Civil_Twilight       0
dtype: int64

In [9]:
# There are over 100 types of weather conditions. We only want the most common. 

condition_value_counts = df['Weather_Condition'].value_counts()
condition_value_counts


Fair                                   1097777
Mostly Cloudy                           362226
Cloudy                                  346177
Partly Cloudy                           248337
Clear                                   172509
                                        ...   
Heavy Rain Showers                           1
Light Haze                                   1
Heavy Thunderstorms with Small Hail          1
Sand / Dust Whirlwinds / Windy               1
Thunder and Hail / Windy                     1
Name: Weather_Condition, Length: 125, dtype: int64

In [10]:
# Drop all columns with weather conditions under 600 occurrences
condition_value_counts = condition_value_counts[condition_value_counts > 600]
df = df[df['Weather_Condition'].isin(condition_value_counts.index)]

In [11]:
# Some of the weather conditions mean the same thing ("Rain Shower" and "Rain Showers", for example.)

# Fair <- Clear, Fair / Windy, N/A Precipitation
# Cloudy <- Overcast, Cloudy / Windy, Mostly Cloudy, Mostly Cloudy / Windy
# Partly Cloudy <- Scattered Clouds, Partly Cloudy / Windy
# Rain <- Heavy Rain, Rain / Windy, Heavy Rain / Windy
# Light Rain <- Mist, Light Drizzle, Light Rain / Windy, Drizzle, Showers in the Vicinity
# Snow <- Heavy Snow
# Light Snow <- Light Snow / Windy
# Wintry Mix <- Light Freezing Rain
# Thunderstorm <- T-Storm, Heavy T-Storm
# Light Thunderstorm <- Thunder in the Vicinity, Thunder, Light Rain with Thunder, Light Thunderstorms and Rain
# Fog <- Haze, Patches of Fog, Shallow Fog, Haze / Windy, Fog / Windy
# Smoke

weather_lookup = {
    'Fair': 'Fair',
    'Clear': 'Fair',
    'Fair / Windy': 'Fair',
    'N/A Precipitation': 'Fair',
    'Cloudy': 'Cloudy',
    'Overcast': 'Cloudy',
    'Cloudy / Windy': 'Cloudy',
    'Mostly Cloudy': 'Cloudy',
    'Mostly Cloudy / Windy': 'Cloudy',
    'Partly Cloudy': 'Partly Cloudy',
    'Scattered Clouds': 'Partly Cloudy',
    'Partly Cloudy / Windy': 'Partly Cloudy',
    'Rain': 'Rain',
    'Heavy Rain': 'Rain',
    'Rain / Windy': 'Rain',
    'Heavy Rain / Windy': 'Rain',
    'Light Rain': 'Light Rain',
    'Mist': 'Light Rain',
    'Light Drizzle': 'Light Rain',
    'Light Rain / Windy': 'Light Rain',
    'Drizzle': 'Light Rain',
    'Showers in the Vicinity': 'Light Rain',
    'Snow': 'Snow',
    'Heavy Snow': 'Snow',
    'Light Snow': 'Light Snow',
    'Light Snow / Windy': 'Light Snow',
    'Wintry Mix': 'Wintry Mix',
    'Light Freezing Rain': 'Wintry Mix',
    'Thunderstorm': 'Thunderstorm',
    'T-Storm': 'Thunderstorm',
    'Heavy T-Storm': 'Thunderstorm',
    'Light Thunderstorm': 'Light Thunderstorm',
    'Thunder in the Vicinity': 'Light Thunderstorm',
    'Thunder': 'Light Thunderstorm',
    'Light Rain with Thunder': 'Light Thunderstorm',
    'Light Thunderstorms and Rain': 'Light Thunderstorm',
    'Fog': 'Fog',
    'Haze': 'Fog',
    'Patches of Fog': 'Fog',
    'Shallow Fog': 'Fog',
    'Haze / Windy': 'Fog',
    'Fog / Windy': 'Fog',
    'Smoke': 'Smoke'
}

def weather_map(weather_condition):
    return weather_lookup[weather_condition]


df['Weather_Condition'] = df.Weather_Condition.map(weather_map)
df['Weather_Condition'].value_counts()

Fair                  1286300
Cloudy                 805583
Partly Cloudy          297024
Light Rain             141513
Fog                     79013
Light Snow              45603
Rain                    44334
Light Thunderstorm      19181
Thunderstorm            11067
Smoke                    7171
Snow                     6656
Wintry Mix               4732
Name: Weather_Condition, dtype: int64